In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/Colab Notebooks/styleTransfer/LinearStyleTransfer

/content/drive/My Drive/Colab Notebooks/styleTransfer/LinearStyleTransfer


In [3]:
#!pip install torch==1.4.0 torchvision==0.5.0

     |█████████▉                      | 232.5MB 1.5MB/s eta 0:05:58
ERROR: Operation cancelled by user


In [4]:
import os
import torch
from PIL import Image
from libs.Loader import Dataset
from libs.Matrix import MulLayer
import torch.backends.cudnn as cudnn
from libs.models import encoder3,encoder4
from libs.models import decoder3,decoder4
import torchvision.transforms as transforms
from libs.utils import makeVideo, print_options
import time

In [5]:
class Opt(object):
  def __init__(self):
    self.vgg_dir = 'models/vgg_r31.pth' # pre-trained encoder path
    self.decoder_dir = 'models/dec_r31.pth' # pre-trained decoder path
    self.matrix_dir = "models/r31.pth" # path to pre-trained model
    self.style = "data/style/2.jpeg" # path to style image
    self.content_dir = "data/content/buildings/" # path to video frames
    self.loadSize = 512 # scale image size
    self.fineSize = 512 # crop image size
    self.name = "transferred_video" # name of generated video
    self.layer = "r31" # features of which layer to transform
    self.outf = "output/" # output folder


In [6]:
# PREPARATIONS
opt = Opt()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print_options(opt)

os.makedirs(opt.outf,exist_ok=True)
cudnn.benchmark = True

----------------- Options ---------------
              content_dir: data/content/buildings/       
              decoder_dir: models/dec_r31.pth            
                 fineSize: 512                           
                    layer: r31                           
                 loadSize: 512                           
               matrix_dir: models/r31.pth                
                     name: transferred_video             
                     outf: output/                       
                    style: data/style/2.jpeg             
                  vgg_dir: models/vgg_r31.pth            
----------------- End -------------------


In [7]:
# DATA
def loadImg(imgPath):
    img = Image.open(imgPath).convert('RGB')
    transform = transforms.Compose([
                transforms.Resize(opt.loadSize),
                transforms.ToTensor()])
    return transform(img)
styleV = loadImg(opt.style).unsqueeze(0).to(device)

content_dataset = Dataset(opt.content_dir,
              loadSize = opt.loadSize,
              fineSize = opt.fineSize,
              test = True,
              video = True)
content_loader = torch.utils.data.DataLoader(dataset = content_dataset,
					              batch_size = 1,
				 	              shuffle = False)

In [8]:
# MODEL
if(opt.layer == 'r31'):
    vgg = encoder3()
    dec = decoder3()
elif(opt.layer == 'r41'):
    vgg = encoder4()
    dec = decoder4()
matrix = MulLayer(layer=opt.layer)

vgg.load_state_dict(torch.load(opt.vgg_dir))
dec.load_state_dict(torch.load(opt.decoder_dir))
matrix.load_state_dict(torch.load(opt.matrix_dir, map_location=device))
vgg.to(device)
dec.to(device)
matrix.to(device)

MulLayer(
  (snet): CNN(
    (convs): Sequential(
      (0): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (fc): Linear(in_features=1024, out_features=1024, bias=True)
  )
  (cnet): CNN(
    (convs): Sequential(
      (0): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (fc): Linear(in_features=1024, out_features=1024, bias=True)
  )
  (compress): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
  (unzip): Conv2d(32, 256, kernel_size=(1, 1), stride=(1, 1))
)

In [9]:
# GLOBAL VARIABLE
contentV = torch.Tensor(1,3,opt.fineSize,opt.fineSize).to(device)

In [10]:
# TRANSFER
result_frames = []
contents = []
style = styleV.squeeze(0).cpu().numpy()
sF = vgg(styleV)

for i,(content,contentName) in enumerate(content_loader):
    # print('Transfer frame %d...'%i)
    start = time.time()
    contentName = contentName[0]
    contentV.resize_(content.size()).copy_(content)
    contents.append(content.squeeze(0).float().numpy())
    # forward
    with torch.no_grad():
        cF = vgg(contentV)

        if(opt.layer == 'r41'):
            feature,transmatrix = matrix(cF[opt.layer],sF[opt.layer])
        else:
            feature,transmatrix = matrix(cF,sF)
        transfer = dec(feature)

    transfer = transfer.clamp(0,1)
    result_frames.append(transfer.squeeze(0).cpu().numpy())
    end = time.time()
    print('fps: %.2f' % (1 / (end-start)))

outf_name = '{:s}/{:s}_stylized_{:s}{:s}'.format(
        opt.outf, 'content_vs_transfer', os.path.splitext(os.path.basename(opt.style))[0], '.avi'
    )
makeVideo(contents,style,result_frames,outf_name)

fps: 0.52
fps: 16.25
fps: 16.55
fps: 12.52
fps: 9.99
fps: 11.67
fps: 9.51
fps: 9.84
fps: 10.03
fps: 10.57
fps: 10.58
fps: 9.45
fps: 11.46
fps: 11.63
fps: 11.90
fps: 12.14
fps: 12.13
fps: 9.44
fps: 11.64
fps: 11.94
fps: 12.10
fps: 12.13
fps: 9.43
fps: 11.64
fps: 10.58
fps: 9.43
fps: 9.45
fps: 11.64
fps: 11.97
fps: 12.09
fps: 9.46
fps: 11.67
fps: 11.94
fps: 12.14
fps: 9.46
fps: 11.11
fps: 11.98
fps: 12.15
fps: 9.46
fps: 11.64
fps: 12.14
fps: 12.11
fps: 9.42
fps: 11.63
fps: 12.11
fps: 9.46
fps: 11.46
fps: 12.01
fps: 11.95
fps: 12.11
fps: 9.80
fps: 10.23
fps: 11.65
fps: 9.41
fps: 11.61
fps: 12.16
fps: 12.14
fps: 9.44
fps: 9.45
fps: 10.57
Stack transferred frames back to video...
Transferred video saved at output//content_vs_transfer_stylized_2.avi.
